<a href="https://colab.research.google.com/github/YAMINISARASWATHI04/LyricsGeneration/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Importing Required Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [ ]:
file_path = "/content/drive/MyDrive/Dakshinagiri/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.train.tsv"

In [ ]:
# @title Parsing the data
input_texts, target_texts = [], []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        eng, mar = line.strip().split('\t')[:2]
        input_texts.append(eng)
        target_texts.append('\t' + mar + '\n')  # Add start/end tokens
print (input_texts)
print(target_texts)

['अँग्री', 'अँजिओग्राफी', 'अँजिओग्राफी', 'अँजिओग्राफी', 'अँजिओग्राफी', 'अँटिल्स', 'अँटिल्स', 'अँटिल्स', 'अँटिल्स', 'अँटिल्स', 'अँटिल्स', 'अँटिल्स', 'अँटी', 'अँटी', 'अँटी', 'अँटी', 'अँटी', 'अँड', 'अँडर्स', 'अँडर्स', 'अँडर्स', 'अँडर्स', 'अँडर्स', 'अँडर्स', 'अँडर्स', 'अँडर्स', 'अँड्रॉइड', 'अँथनी', 'अँप', 'अंक', 'अंकांत', 'अंकांत', 'अंकाचा', 'अंकाचे', 'अंकाच्या', 'अंकाच्या', 'अंकात', 'अंकित', 'अंकी', 'अंकी', 'अंकुश', 'अंग', 'अंग', 'अंगच्या', 'अंगठा', 'अंगठी', 'अंगठी', 'अंगठे', 'अंगठे', 'अंगणवाडी', 'अंगणवाडी', 'अंगणवाडी', 'अंगणात', 'अंगणी', 'अंगणी', 'अंगद', 'अंगदुखी', 'अंगदुखी', 'अंगभूत', 'अंगभूत', 'अंगविक्षेप', 'अंगविक्षेप', 'अंगविक्षेप', 'अंगांनी', 'अंगांनी', 'अंगाचा', 'अंगात', 'अंगात', 'अंगातील', 'अंगातील', 'अंगाने', 'अंगारा', 'अंगारा', 'अंगाला', 'अंगावर', 'अंगावर', 'अंगावरची', 'अंगावरची', 'अंगावरची', 'अंगावरील', 'अंगावरील', 'अंगी', 'अंगी', 'अंगे', 'अंघोळ', 'अंजन', 'अंजनी', 'अंजनी', 'अंजनेरी', 'अंजनेरी', 'अंजली', 'अंजली', 'अंजली', 'अंजली', 'अंजली', 'अंड', 'अंड', 'अंडर', 'अंडर', 'अंडर', '

In [ ]:
# @title Tokenizing the data
input_chars = sorted(set("".join(input_texts)))
target_chars = sorted(set("".join(target_texts)))

input_token_index = {char: i+1 for i, char in enumerate(input_chars)}  # reserve 0 for padding
target_token_index = {char: i+1 for i, char in enumerate(target_chars)}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

input_vocab_size = len(input_token_index) + 1
target_vocab_size = len(target_token_index) + 1
print(input_chars)
print(target_chars)
print(input_token_index)
print(target_token_index)


['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऍ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ऱ', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॲ', '\u200d']
['\t', '\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{'ँ': 1, 'ं': 2, 'ः': 3, 'अ': 4, 'आ': 5, 'इ': 6, 'ई': 7, 'उ': 8, 'ऊ': 9, 'ऋ': 10, 'ऍ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'च': 21, 'छ': 22, 'ज': 23, 'झ': 24, 'ञ': 25, 'ट': 26, 'ठ': 27, 'ड': 28, 'ढ': 29, 'ण': 30, 'त': 31, 'थ': 32, 'द': 33, 'ध': 34, 'न': 35, 'प': 36, 'फ': 37, 'ब': 38, 'भ': 39, 'म': 40, 'य': 41, 'र': 42, 'ऱ': 43, 'ल': 44, 'ळ': 45, 'व': 46, 'श': 47, 'ष': 48, 'स': 49, 'ह': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ'

In [ ]:
# @title Format Dataset
encoder_input_data = [[input_token_index[c] for c in seq] for seq in input_texts]
decoder_input_data = [[target_token_index[c] for c in seq] for seq in target_texts]
decoder_target_data = [seq[1:] for seq in decoder_input_data]

max_encoder_seq_length = max(len(seq) for seq in encoder_input_data)
max_decoder_seq_length = max(len(seq) for seq in decoder_input_data)

encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_decoder_seq_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_decoder_seq_length, padding='post')

decoder_target_data_onehot = tf.keras.utils.to_categorical(decoder_target_data, num_classes=target_vocab_size)



In [ ]:
def build_seq2seq_model(cell_type='LSTM', embedding_dim=64, hidden_dim=128, num_layers=1):
    RNN = {'RNN': SimpleRNN, 'LSTM': LSTM, 'GRU': GRU}[cell_type]

    # Encoder
    encoder_inputs = Input(shape=(None,))
    enc_embed = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)

    for i in range(num_layers):
        return_seq = (i != num_layers - 1)
        if i == 0:
            encoder_output = RNN(hidden_dim, return_sequences=return_seq, return_state=True)
            if cell_type == 'LSTM':
                _, state_h, state_c = encoder_output(enc_embed)
                encoder_states = [state_h, state_c]
            else:
                _, state_h = encoder_output(enc_embed)
                encoder_states = [state_h]
        else:
            enc_embed = RNN(hidden_dim, return_sequences=return_seq)(enc_embed)

    # Decoder
    decoder_inputs = Input(shape=(None,))
    dec_embed = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)

    for i in range(num_layers):
        return_seq = True
        if i == 0:
            decoder_rnn = RNN(hidden_dim, return_sequences=return_seq, return_state=True)
            if cell_type == 'LSTM':
                decoder_outputs, _, _ = decoder_rnn(dec_embed, initial_state=encoder_states)
            else:
                decoder_outputs, _ = decoder_rnn(dec_embed, initial_state=encoder_states)
        else:
            dec_embed = RNN(hidden_dim, return_sequences=return_seq)(dec_embed)

    decoder_dense = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model


In [ ]:
model = build_seq2seq_model(cell_type='LSTM', embedding_dim=128, hidden_dim=256, num_layers=1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data_onehot,
    batch_size=64,
    epochs=30,
    validation_split=0.2
)


Epoch 1/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.7350 - loss: 0.9820 - val_accuracy: 0.7599 - val_loss: 0.9282
Epoch 2/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.7963 - loss: 0.6650 - val_accuracy: 0.7852 - val_loss: 0.8632
Epoch 3/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.8575 - loss: 0.4564 - val_accuracy: 0.8327 - val_loss: 0.6913
Epoch 4/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9125 - loss: 0.2833 - val_accuracy: 0.8663 - val_loss: 0.5531
Epoch 5/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9376 - loss: 0.2020 - val_accuracy: 0.8768 - val_loss: 0.5144
Epoch 6/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9480 - loss: 0.1646 - val_accuracy: 0.8890 - val_loss: 0.4679
Epoch 7/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9534 - loss: 0.1429 - val_accuracy: 0.8932 - val_loss: 0.4257
Epoch 8/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9564 - loss: 0.1300 - val_a

In [ ]:
from tensorflow import keras # Import keras from tensorflow


In [ ]:
# Encoder inference model
encoder_inputs = model.input[0]  # Input to encoder
# Get the correct LSTM layer name - it's likely 'lstm_16' based on the error message
encoder_outputs, state_h_enc, state_c_enc = model.get_layer("lstm").output  # Changed 'lstm' to 'lstm_16'
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

latent_dim = encoder_states[0].shape[-1]

decoder_inputs = model.input[1]  # Input to decoder
# Pass decoder inputs through embedding layer
dec_embed = model.get_layer("embedding_1")(decoder_inputs) # This line was added

decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_h")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_c")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Similarly, adjust the decoder LSTM layer name - likely 'lstm_17'
decoder_lstm = model.get_layer("lstm_1")  # Changed 'lstm_1' to 'lstm_17'
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    dec_embed, initial_state=decoder_states_inputs # decoder_inputs was changed to dec_embed
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.get_layer("dense")  # Or "dense_18" if that's the actual name from the error
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [ ]:
# @title Decoding Test Sentences
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence with just the start character
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']

    # Store the decoded characters
    decoded_sentence = ""

    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            break

        # Update the target sequence
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence



In [ ]:
# @title Evaluating the Model
# Evaluate on test split
X_train, X_test, y_train, y_test = train_test_split(
    encoder_input_data, decoder_input_data, test_size=0.1, random_state=42)

# Predict sample
for i in range(5):
    input_seq = X_test[i:i+1]
    decoded = decode_sequence(input_seq)
    print("Input:", input_texts[i])
    print("Predicted:", decoded)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Input: अँग्री
Predicted: romeeyo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Input: अँजिओग्राफी
Predicted: pullingi

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

In [ ]:
def load_test_data(filepath):
    test_pairs = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if '\t' in line:
                # Splitting the line by tabs and taking only the first two elements
                parts = line.split('\t')
                input_text, target_text = parts[:2]  # Assign the first two elements to input_text and target_text
                target_text = '\t' + target_text + '\n'  # for seq2seq start/end tokens
                test_pairs.append((input_text, target_text))
    return test_pairs
test_pairs = load_test_data('/content/drive/MyDrive/Dakshinagiri/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.test.tsv')  # ✅ update with your correct path
test_input_texts = [inp for inp, _ in test_pairs]
test_target_texts = [tgt for _, tgt in test_pairs]


In [ ]:
embedding_dim = 64
hidden_dim = 256
cell_type = 'LSTM'
num_layers = 1

# Load test data


In [ ]:
!pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [ ]:
tokenizer_input = Tokenizer()
tokenizer_input.fit_on_texts(input_texts)  # Fit on training data

tokenizer_output = Tokenizer(char_level=True)  # char_level=True for decoder
tokenizer_output.fit_on_texts(target_texts)

In [ ]:
encoder_test_input_data = tokenizer_input.texts_to_sequences(test_input_texts)
encoder_test_input_data = pad_sequences(encoder_test_input_data, maxlen=max_encoder_seq_length, padding='post')

decoder_test_input_data = tokenizer_output.texts_to_sequences([t[:-1] for t in test_target_texts])
decoder_test_input_data = pad_sequences(decoder_test_input_data, maxlen=max_decoder_seq_length, padding='post')

decoder_test_target_data = tokenizer_output.texts_to_sequences([t[1:] for t in test_target_texts])
decoder_test_target_data = pad_sequences(decoder_test_target_data, maxlen=max_decoder_seq_length, padding='post')

# One-hot encode the targets
decoder_test_target_data_onehot = to_categorical(decoder_test_target_data, num_classes=target_vocab_size)

In [ ]:
model = build_seq2seq_model(
    cell_type=cell_type,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(f"\n🔁 Training model with embedding_dim={embedding_dim}, hidden_dim={hidden_dim}...")
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data_onehot,
    batch_size=64,
    epochs=30,
    validation_split=0.2,
    verbose=1
)


🔁 Training model with embedding_dim=64, hidden_dim=256...
Epoch 1/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.7303 - loss: 1.0203 - val_accuracy: 0.7545 - val_loss: 0.9271
Epoch 2/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.7921 - loss: 0.6791 - val_accuracy: 0.7744 - val_loss: 0.9102
Epoch 3/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8254 - loss: 0.5629 - val_accuracy: 0.7862 - val_loss: 0.9031
Epoch 4/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.8651 - loss: 0.4327 - val_accuracy: 0.8148 - val_loss: 0.7976
Epoch 5/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9034 - loss: 0.3107 - val_accuracy: 0.8431 - val_loss: 0.6725
Epoch 6/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - accuracy: 0.9292 - loss: 0.2295 - val_accuracy: 0.8622 - val_loss: 0.5849
Epoch 7/30
704/704 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9426 - loss: 0.1830 - val_accuracy: 0.8740 - val_loss: 0.5423
Epoch 8/30
704/704 ━━━━━━━━━━━━━━━━

In [ ]:
print("\n📊 Evaluating on test set...")
test_loss, test_acc = model.evaluate(
    [encoder_test_input_data, decoder_test_input_data],
    decoder_test_target_data_onehot,
    verbose=1
)

print(f"\n✅ Final Test Accuracy (embedding_dim={embedding_dim}): {test_acc:.4f}")


📊 Evaluating on test set...
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6416 - loss: 3.1891

✅ Final Test Accuracy (embedding_dim=64): 0.6375


In [ ]:
print("\n🔮 Sample Predictions on Test Data:")
for i in range(10):  # Change this to len(test_input_texts) for all
    input_seq = encoder_test_input_data[i:i+1]
    decoded_sentence = decode_sequence(input_seq)

    print(f"{i+1}. Latin: {test_input_texts[i]}")
    print(f"   Predicted: {decoded_sentence}")
    print(f"   Actual:    {test_target_texts[i][1:-1]}")  # remove \t and \n
    print()



🔮 Sample Predictions on Test Data:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1. Latin: अँडरसन
   Predicted: e

   Actual:    andarasan

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2. Latin: अँडरसन
   Predicted: e

   Actual:    andarasana

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3. Latin: अँडरसन
   Predicted: e

   Actual:    andarashana

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
4. Latin: अँडरसन
   Predicted: e

   Actual:    andarsan

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
5. Latin: अँडरसन
   Predicted: e

   Actual:    andarsen

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━